# Optimal Cover Plot
The hope of this project is to generate the “Goldilocks” zone for miRNA cover. To do this, we are using the previously developed “random cover” simulation compared to a newly generated “optimized cover” using a greedy algorithm. These two curves will be plotted together with an X marking each species' actual seed cover. 

For organization purposes, the random cover and optimized cover will be generated separately, with final data being outputted to folders that will be consolidated in the “final cover plot” program. 
Additionally, we will be using three datasets: TargetScan data (which selects only the most represented 3’UTR),  Biomart+consolidation (which considers all 3’UTR and removes repeats), and Biomart+generation (which generates a gene that contains all versions of transcriptID).  Each of these datasets represents different ways miRNA interacts in the cell, therefore comparing the results of each dataset will allow us to determine which method is most effective. 

This script generates the "optimal cover" through a greedy algorithm. For each set of random genes, the algorithm ranks all possible seeds by their frequency. The highest ranking seeds are the selected as the 'optimal set'. The size of the set matches that of the random seeds in order to compare the effective cover of the seeds. 
We repeat this algorithm 30 times from 100 - 1600 seeds in intervals of 100. 

In the optimal_cover_simulate_main() function, we test to see how many genes the optimal seeds cover. The cover is saved in a gene file that will be compared to random seeds. 

In [31]:
from numpy import loadtxt
import os
import glob
import random
from matplotlib import pyplot as plt
import ast
import time
import numpy as np
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [26]:
species_list = [
    ('Lamprey genes (Pmarinus_7.0)','Sea Lamprey (Petromyzon marinus)'),
    ('Hagfish genes (Eburgeri_3.2)','Inshore hagfish (Eptatretus burgeri)'),
    ('Elephant shark genes (Callorhinchus_milii-6.1.3)','Australian ghostshark (Callorhinchus milii)'),
    ('Spotted gar genes (LepOcu1)','Spotted gar (Lepisosteus oculatus)'),
    ('Zebrafish genes (GRCz11)','Zebrafish (Danio rerio)'),
    ('Atlantic cod genes (gadMor3.0)','Cod (Gadus morhua)'),
    ('Coelacanth genes (LatCha1)','Coelacanth (Latimeria chalumnae)'),
    ('Tropical clawed frog genes (Xenopus_tropicalis_v9.1)','Tropical clawed frog (Xenopus tropicalis)'),
    ('Tuatara genes (ASM311381v1)','Tuatara (Sphenodon punctatus)'),
    ('Green anole genes (AnoCar2.0v2)','Green anole lizard (Anolis carolinensis)'),
    ('Painted turtle genes (Chrysemys_picta_bellii-3.0.3)','Western painted turtle (Chrysemys picta bellii)'),
    ('Zebra finch genes (bTaeGut1_v1.p)','Zebra finch (Taeniopygia guttata)'),
    ('Chicken (maternal Broiler) genes (bGalGal1.mat.broiler.GRCg7b)','Chicken (Gallus gallus)'),
    ('Platypus genes (mOrnAna1.p.v1)','Platypus (Ornithorhynchus anatinus)'),
    ('Opossum genes (ASM229v1)','Gray short-tailed opossum (Monodelphis domestica)'),
    ('Armadillo genes (Dasnov3.0)','Nine-banded armadillo (Dasypus novemcinctus)'),
    ('Cow genes (ARS-UCD1.2)','Cow (Bos taurus)'),
    ('Dog genes (ROS_Cfam_1.0)','Dog (Canis familiaris)'),
    ('Rabbit genes (OryCun2.0)','Rabbit (Oryctolagus cuniculus)'),
    ('Guinea Pig genes (Cavpor3.0)','Guinea pig (Cavia porcellus)'),
    ('Rat genes (mRatBN7.2)','Norway rat (Rattus norvegicus)'),
    ('Mouse genes (GRCm39)','House mouse (Mus musculus)'),
    ('Human genes (GRCh38.p13)','Human (Homo sapiens)'),
    ('Human TargetScan','Human (Homo sapiens) TargetScan'),
    ('All','All')
]

In [ ]:
def gen_greedy_set(filename, canon_site, genome, sim_num):
    start_time = time.time()
    
    new_filename = filename+"/"+str(sim_num)+".txt"
    if os.path.exists(new_filename):
        print(str(sim_num)+" exists, skipping to next sim")
        return
    
    genome = random.sample(genome,int(len(genome)*.95))
    cover_list = {}
    if canon_site == 'A':
        consider_A = True
        while len(genome)>0:
            motif_dict = {}
            for gene in genome:
                set_of_motifs = []
                for i in range(len(gene)-6):
                    motif = gene[i:i+7]
                    if consider_A == True:
                        if motif[-1] == 'A':
                            set_of_motifs.append(motif)
                    else:
                        set_of_motifs.append(motif)
                for motif in set_of_motifs:
                    if (motif in motif_dict):
                        motif_dict[motif] += 1
                    else:
                        motif_dict[motif] = 1
            if len(motif_dict) > 0:
                max_motif = max(motif_dict, key = motif_dict.get)
            else:
                consider_A = False
                break
            cover_list[max_motif] = motif_dict[max_motif]
            genome = [ x for x in genome if max_motif not in x ]
        with open(new_filename, "w") as output:
            output.write(str(list(cover_list.keys())))
                

    elif canon_site == 'B':
        while len(genome)>0:
            motif_dict = {}
            for gene in genome:
                set_of_motifs = []
                for i in range(len(gene)-6):
                    set_of_motifs.append(gene[i:i+7])
                set_of_motifs = [*set(set_of_motifs)]
                for motif in set_of_motifs:
                    if (motif in motif_dict):
                        motif_dict[motif] += 1
                    else:
                        motif_dict[motif] = 1
            max_motif = max(motif_dict, key = motif_dict.get)
            cover_list[max_motif] = motif_dict[max_motif]
            genome = [ x for x in genome if max_motif not in x ]
        with open(new_filename, "w") as output:
            output.write(str(list(cover_list.keys())))
                

    elif canon_site == 'C':
    ##It is not guaranteed that 'NNNNNNNA' will cover every gene, further action is required
        consider_A = True
        while len(genome)>0:
            motif_dict = {}
            for gene in genome:
                set_of_motifs = []
                for i in range(len(gene)-7):
                    motif = gene[i:i+8]
                    if consider_A == True:
                        if motif[-1] == 'A':
                            set_of_motifs.append(motif)
                    else:
                        set_of_motifs.append(motif)
                for motif in set_of_motifs:
                    if (motif in motif_dict):
                        motif_dict[motif] += 1
                    else:
                        motif_dict[motif] = 1
            if len(motif_dict) > 0:
                max_motif = max(motif_dict, key = motif_dict.get)
            else:
                consider_A = False
                break
            cover_list[max_motif] = motif_dict[max_motif]
            genome = [ x for x in genome if max_motif not in x ]
        with open(new_filename, "w") as output:
            output.write(str(list(cover_list.keys())))
    print("Trial "+str(sim_num)+" done in: "+" %s seconds" % (time.time() - start_time))

In [17]:
##calculate greedy seed list
def calc_greedy(genome_filename, canon_site):
    if genome_filename == 'All':
        data = []
        for species in species_list:
            if species[0]!='All':
                temp_data = loadtxt("Genome Data/"+species[0]+".txt", comments=">",dtype="str")
                #remove all the Unavailable sequences and sequences that are too short
                allowed_char = 'AGCT'
                temp_data = list(filter(lambda a: all(ch in allowed_char for ch in a) and len(a)>7, temp_data))
                print(species[0]+": "+str(len(temp_data)))
                data = data+temp_data
    else:
        data = loadtxt("Genome Data/"+genome_filename+".txt", comments=">",dtype="str")
    
    #remove all the Unavailable sequences and sequences that are too short
    allowed_char = 'AGCT'
    genome = list(filter(lambda a: all(ch in allowed_char for ch in a) and len(a)>7, data))
    
    new_file_path = "Optimal Seed Set/"+genome_filename+" Seed Set/Canon Site "+canon_site
    if not os.path.exists(new_file_path):
        os.makedirs(new_file_path)
    
    for i in range(0,15):
        gen_greedy_set(new_file_path, canon_site, genome.copy(), i)

In [18]:
#run calc_greedy() here
canon_sites = ['A','C','B']
for canon_site in canon_sites:
    for species in species_list:
        calc_greedy(species[0], canon_site)

0 exists, skipping to next sim
1 exists, skipping to next sim
2 exists, skipping to next sim
3 exists, skipping to next sim
4 exists, skipping to next sim
5 exists, skipping to next sim
6 exists, skipping to next sim
7 exists, skipping to next sim
8 exists, skipping to next sim
9 exists, skipping to next sim
10 exists, skipping to next sim
11 exists, skipping to next sim
12 exists, skipping to next sim
13 exists, skipping to next sim
14 exists, skipping to next sim
0 exists, skipping to next sim
1 exists, skipping to next sim
2 exists, skipping to next sim
3 exists, skipping to next sim
4 exists, skipping to next sim
5 exists, skipping to next sim
6 exists, skipping to next sim
7 exists, skipping to next sim
8 exists, skipping to next sim
9 exists, skipping to next sim
10 exists, skipping to next sim
11 exists, skipping to next sim
12 exists, skipping to next sim
13 exists, skipping to next sim
14 exists, skipping to next sim
0 exists, skipping to next sim
1 exists, skipping to next si

0 exists, skipping to next sim
1 exists, skipping to next sim
2 exists, skipping to next sim
3 exists, skipping to next sim
4 exists, skipping to next sim
5 exists, skipping to next sim
6 exists, skipping to next sim
7 exists, skipping to next sim
8 exists, skipping to next sim
9 exists, skipping to next sim
10 exists, skipping to next sim
11 exists, skipping to next sim
12 exists, skipping to next sim
13 exists, skipping to next sim
14 exists, skipping to next sim
0 exists, skipping to next sim
1 exists, skipping to next sim
2 exists, skipping to next sim
3 exists, skipping to next sim
4 exists, skipping to next sim
5 exists, skipping to next sim
6 exists, skipping to next sim
7 exists, skipping to next sim
8 exists, skipping to next sim
9 exists, skipping to next sim
10 exists, skipping to next sim
11 exists, skipping to next sim
12 exists, skipping to next sim
13 exists, skipping to next sim
14 exists, skipping to next sim
0 exists, skipping to next sim
1 exists, skipping to next si

Trial 10 done in:  25.704041004180908 seconds
Trial 11 done in:  25.751364946365356 seconds
Trial 12 done in:  25.82005023956299 seconds
Trial 13 done in:  25.61865997314453 seconds
Trial 14 done in:  25.86090922355652 seconds
Trial 0 done in:  2.8996798992156982 seconds
Trial 1 done in:  2.8858659267425537 seconds
Trial 2 done in:  2.8560640811920166 seconds
Trial 3 done in:  2.8528549671173096 seconds
Trial 4 done in:  2.857025146484375 seconds
Trial 5 done in:  2.8696200847625732 seconds
Trial 6 done in:  2.8718671798706055 seconds
Trial 7 done in:  2.843534231185913 seconds
Trial 8 done in:  2.8847920894622803 seconds
Trial 9 done in:  2.857527017593384 seconds
Trial 10 done in:  2.8823130130767822 seconds
Trial 11 done in:  2.8775057792663574 seconds
Trial 12 done in:  2.8267431259155273 seconds
Trial 13 done in:  2.862366199493408 seconds
Trial 14 done in:  2.8608059883117676 seconds
Trial 0 done in:  32.87644004821777 seconds
Trial 1 done in:  32.79857921600342 seconds
Trial 2 d

Trial 13 done in:  27.616547107696533 seconds
Trial 14 done in:  27.686988830566406 seconds
Trial 0 done in:  20.103928089141846 seconds
Trial 1 done in:  20.133544921875 seconds
Trial 2 done in:  20.45239806175232 seconds
Trial 3 done in:  20.4511981010437 seconds
Trial 4 done in:  20.34689211845398 seconds
Trial 5 done in:  20.39083695411682 seconds
Trial 6 done in:  20.413729906082153 seconds
Trial 7 done in:  20.543954849243164 seconds
Trial 8 done in:  20.305716037750244 seconds
Trial 9 done in:  20.462982892990112 seconds
Trial 10 done in:  20.487444162368774 seconds
Trial 11 done in:  20.393539905548096 seconds
Trial 12 done in:  20.21811032295227 seconds
Trial 13 done in:  20.415398120880127 seconds
Trial 14 done in:  20.444629192352295 seconds
Trial 0 done in:  12.018994092941284 seconds
Trial 1 done in:  11.975967168807983 seconds
Trial 2 done in:  12.072570085525513 seconds
Trial 3 done in:  12.067365884780884 seconds
Trial 4 done in:  12.021875143051147 seconds
Trial 5 done

Trial 11 done in:  5.828848838806152 seconds
Trial 12 done in:  5.830526113510132 seconds
Trial 13 done in:  5.806433916091919 seconds
Trial 14 done in:  5.833518981933594 seconds
Trial 0 done in:  12.517122983932495 seconds
Trial 1 done in:  12.454100847244263 seconds
Trial 2 done in:  12.403861999511719 seconds
Trial 3 done in:  12.455451965332031 seconds
Trial 4 done in:  12.521054983139038 seconds
Trial 5 done in:  12.44011902809143 seconds
Trial 6 done in:  12.456623077392578 seconds
Trial 7 done in:  12.41594386100769 seconds
Trial 8 done in:  12.43298077583313 seconds
Trial 9 done in:  12.497072696685791 seconds
Trial 10 done in:  12.426936864852905 seconds
Trial 11 done in:  12.478798151016235 seconds
Trial 12 done in:  12.413766860961914 seconds
Trial 13 done in:  12.49167799949646 seconds
Trial 14 done in:  12.49067997932434 seconds
Trial 0 done in:  13.977410078048706 seconds
Trial 1 done in:  13.997817277908325 seconds
Trial 2 done in:  14.230617046356201 seconds
Trial 3 do

Trial 14 done in:  8.124279022216797 seconds
Trial 0 done in:  13.121939897537231 seconds
Trial 1 done in:  13.132247924804688 seconds
Trial 2 done in:  13.139892816543579 seconds
Trial 3 done in:  13.154759883880615 seconds
Trial 4 done in:  13.131334066390991 seconds
Trial 5 done in:  13.14099407196045 seconds
Trial 6 done in:  13.160394191741943 seconds
Trial 7 done in:  13.13718295097351 seconds
Trial 8 done in:  13.093009948730469 seconds
Trial 9 done in:  13.123486995697021 seconds
Trial 10 done in:  13.122497081756592 seconds
Trial 11 done in:  13.128784894943237 seconds
Trial 12 done in:  13.103846788406372 seconds
Trial 13 done in:  13.145153999328613 seconds
Trial 14 done in:  13.154445886611938 seconds
Trial 0 done in:  22.58599305152893 seconds
Trial 1 done in:  22.663052082061768 seconds
Trial 2 done in:  22.66863989830017 seconds
Trial 3 done in:  22.728867769241333 seconds
Trial 4 done in:  22.77083420753479 seconds
Trial 5 done in:  20.38983917236328 seconds
Trial 6 don

In [20]:
def simulate_number(data, number, filename, new_file_name, canon_site):     
    data_for_trials = []
    for trials in range(0,15):
        motifs = []
        file_path = "Optimal Seed Set/"+filename+" Seed Set/Canon Site "+canon_site+"/"+str(trials)+".txt"
        if not os.path.exists(file_path):
            print("Trial "+str(trials)+" does not exist, skipping to next trial")
            break
        with open(file_path, 'r') as f:
            motifs = ast.literal_eval(f.read())
        
        motifs_subset = motifs[0:number]
        covered_list = []

        for gene in data:
            for motif in motifs_subset:
                if motif in gene:
                    covered_list.append(gene)
                    break
                    
        #Remove repeats
        covered_list = [*set(covered_list)]
        #Count the number of genes covered
        covered_num = len(covered_list)

        data_for_trials.append(covered_num)
    print(data_for_trials)
    
    with open(new_file_name, "w") as output:
        output.write(str(data_for_trials))

In [61]:
#for each 100 seeds, count how many genes it covers
def optimal_cover_simulate_main(genome_filename, canon_site):
    start_time = time.time()
    
    if genome_filename == 'All':
        data = []
        for species in species_list:
            if species[0]!='All':
                temp_data = loadtxt("Genome Data/"+species[0]+".txt", comments=">",dtype="str")
                #remove all the Unavailable sequences and sequences that are too short
                allowed_char = 'AGCT'
                temp_data = list(filter(lambda a: all(ch in allowed_char for ch in a) and len(a)>7, temp_data))
                print(species[0]+": "+str(len(temp_data)))
                data = data+temp_data
    else:
        data = loadtxt("Genome Data/"+genome_filename+".txt", comments=">",dtype="str")
    
    #remove all the Unavailable sequences and sequences that are too short
    allowed_char = 'AGCT'
    data = list(filter(lambda a: all(ch in allowed_char for ch in a) and len(a)>7, data))
    
    new_file_path = "Simulated Optimal Coverage/"+genome_filename+"_simulation/Canon Site "+canon_site
    if not os.path.exists(new_file_path):
        os.makedirs(new_file_path)
    
    #loop to simulate 
    for list_size in range(0, 100, 10):
        new_file_name = os.path.join(new_file_path, str(list_size)+".txt")
        if not os.path.exists(new_file_name):
            simulate_number(data, list_size, genome_filename, new_file_name, canon_site)
        else:
            print(new_file_name +" found, skipping to next file")
    for list_size in range(100, 1400, 100):
        new_file_name = os.path.join(new_file_path, str(list_size)+".txt")
        if not os.path.exists(new_file_name):
            simulate_number(data, list_size, genome_filename, new_file_name, canon_site)
        else:
            print(new_file_name +" found, skipping to next file")
    
    print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#run optimal_cover_simulate_main() here
canon_sites = ['A','C','B']
for canon_site in canon_sites:
    for species in species_list:
        optimal_cover_simulate_main(species[0], canon_site)

Simulated Optimal Coverage/Lamprey genes (Pmarinus_7.0)_simulation/Canon Site A/0.txt found, skipping to next file
Simulated Optimal Coverage/Lamprey genes (Pmarinus_7.0)_simulation/Canon Site A/10.txt found, skipping to next file
Simulated Optimal Coverage/Lamprey genes (Pmarinus_7.0)_simulation/Canon Site A/20.txt found, skipping to next file
Simulated Optimal Coverage/Lamprey genes (Pmarinus_7.0)_simulation/Canon Site A/30.txt found, skipping to next file
Simulated Optimal Coverage/Lamprey genes (Pmarinus_7.0)_simulation/Canon Site A/40.txt found, skipping to next file
Simulated Optimal Coverage/Lamprey genes (Pmarinus_7.0)_simulation/Canon Site A/50.txt found, skipping to next file
Simulated Optimal Coverage/Lamprey genes (Pmarinus_7.0)_simulation/Canon Site A/60.txt found, skipping to next file
Simulated Optimal Coverage/Lamprey genes (Pmarinus_7.0)_simulation/Canon Site A/70.txt found, skipping to next file
Simulated Optimal Coverage/Lamprey genes (Pmarinus_7.0)_simulation/Canon 

Simulated Optimal Coverage/Spotted gar genes (LepOcu1)_simulation/Canon Site A/0.txt found, skipping to next file
[7651, 7651, 7651, 7651, 7651, 7651, 7651, 7651, 7651, 7651, 7659, 7651, 7659, 7651, 7659]
[7986, 7983, 7977, 7985, 7985, 7977, 7996, 7972, 7980, 7961, 7986, 7981, 7981, 7969, 7975]
[8156, 8180, 8169, 8172, 8164, 8151, 8162, 8164, 8168, 8153, 8156, 8157, 8144, 8160, 8157]
[8285, 8294, 8279, 8294, 8289, 8291, 8297, 8285, 8283, 8289, 8285, 8280, 8278, 8291, 8281]
[8382, 8390, 8370, 8382, 8377, 8373, 8387, 8382, 8377, 8381, 8373, 8384, 8378, 8381, 8376]
[8450, 8465, 8447, 8460, 8448, 8445, 8460, 8449, 8451, 8459, 8441, 8451, 8447, 8453, 8448]
[8509, 8517, 8507, 8521, 8509, 8508, 8520, 8505, 8500, 8508, 8499, 8504, 8503, 8510, 8500]
[8552, 8562, 8554, 8566, 8554, 8553, 8567, 8549, 8548, 8552, 8549, 8550, 8548, 8555, 8543]
[8594, 8597, 8596, 8604, 8592, 8590, 8606, 8586, 8589, 8593, 8594, 8587, 8591, 8594, 8579]
Simulated Optimal Coverage/Spotted gar genes (LepOcu1)_simulation/C

[7036, 7031, 7050, 7041, 7040, 7035, 7041, 7038, 7034, 7042, 7039, 7039, 7040, 7032, 7039]
[7073, 7075, 7092, 7080, 7082, 7072, 7085, 7080, 7073, 7082, 7082, 7082, 7078, 7078, 7079]
[7105, 7108, 7127, 7108, 7119, 7109, 7116, 7116, 7109, 7116, 7116, 7111, 7112, 7114, 7111]
[7134, 7138, 7154, 7138, 7152, 7138, 7146, 7142, 7139, 7145, 7146, 7141, 7145, 7144, 7141]
Simulated Optimal Coverage/Coelacanth genes (LatCha1)_simulation/Canon Site A/100.txt found, skipping to next file
Simulated Optimal Coverage/Coelacanth genes (LatCha1)_simulation/Canon Site A/200.txt found, skipping to next file
Simulated Optimal Coverage/Coelacanth genes (LatCha1)_simulation/Canon Site A/300.txt found, skipping to next file
Simulated Optimal Coverage/Coelacanth genes (LatCha1)_simulation/Canon Site A/400.txt found, skipping to next file
Simulated Optimal Coverage/Coelacanth genes (LatCha1)_simulation/Canon Site A/500.txt found, skipping to next file
Simulated Optimal Coverage/Coelacanth genes (LatCha1)_simulat

Simulated Optimal Coverage/Painted turtle genes (Chrysemys_picta_bellii-3.0.3)_simulation/Canon Site A/0.txt found, skipping to next file
[9403, 9403, 9403, 9403, 9403, 9403, 9403, 9403, 9403, 9403, 9403, 9403, 9403, 9403, 9403]
[10190, 10202, 10191, 10195, 10191, 10192, 10198, 10194, 10200, 10198, 10167, 10198, 10193, 10198, 10202]
[10472, 10470, 10470, 10474, 10479, 10465, 10472, 10476, 10473, 10473, 10458, 10474, 10466, 10473, 10464]
[10643, 10620, 10617, 10626, 10638, 10628, 10632, 10641, 10621, 10642, 10618, 10635, 10626, 10630, 10616]
[10740, 10721, 10716, 10723, 10734, 10739, 10734, 10738, 10726, 10752, 10730, 10721, 10732, 10736, 10728]
[10813, 10804, 10796, 10798, 10813, 10817, 10797, 10799, 10805, 10817, 10799, 10802, 10800, 10812, 10812]
[10873, 10859, 10860, 10857, 10867, 10874, 10863, 10854, 10862, 10876, 10867, 10864, 10857, 10862, 10871]
[10919, 10917, 10910, 10912, 10918, 10921, 10910, 10907, 10908, 10917, 10919, 10916, 10904, 10907, 10918]
[10964, 10956, 10952, 10958, 

Simulated Optimal Coverage/Platypus genes (mOrnAna1.p.v1)_simulation/Canon Site A/0.txt found, skipping to next file
[9668, 9668, 9668, 9668, 9668, 9668, 9668, 9668, 9668, 9668, 9668, 9668, 9668, 9668, 9668]
[10479, 10482, 10475, 10436, 10465, 10479, 10479, 10467, 10465, 10467, 10467, 10467, 10467, 10479, 10479]
[10809, 10794, 10804, 10799, 10809, 10822, 10809, 10805, 10809, 10809, 10809, 10808, 10809, 10809, 10805]
[10994, 11002, 10998, 10993, 11004, 10998, 10991, 10995, 10993, 10997, 10991, 11007, 11004, 10994, 10999]
[11129, 11150, 11133, 11141, 11142, 11138, 11137, 11129, 11135, 11125, 11132, 11140, 11143, 11133, 11128]
[11237, 11256, 11241, 11234, 11238, 11236, 11237, 11240, 11237, 11231, 11234, 11239, 11248, 11232, 11231]
[11314, 11336, 11313, 11311, 11322, 11318, 11317, 11319, 11318, 11313, 11323, 11320, 11327, 11314, 11298]
[11381, 11402, 11378, 11379, 11382, 11386, 11379, 11385, 11381, 11376, 11389, 11385, 11390, 11381, 11371]
[11443, 11453, 11446, 11439, 11444, 11447, 11440, 

...

[4064, 4190, 4064, 4190, 4148, 4119, 4190, 4190, 4121, 4187, 4190, 4190, 4190, 4190, 4190]
[4607, 4661, 4607, 4656, 4628, 4635, 4655, 4660, 4633, 4656, 4652, 4661, 4655, 4660, 4660]
[4865, 4889, 4855, 4883, 4864, 4871, 4886, 4891, 4866, 4887, 4874, 4883, 4890, 4890, 4890]
[5021, 5038, 5011, 5030, 5022, 5036, 5036, 5040, 5029, 5032, 5037, 5038, 5037, 5032, 5032]
[5134, 5147, 5130, 5139, 5136, 5146, 5139, 5151, 5142, 5145, 5133, 5144, 5144, 5141, 5148]
[5212, 5221, 5217, 5215, 5217, 5225, 5224, 5229, 5217, 5230, 5216, 5226, 5224, 5224, 5223]
[5283, 5294, 5290, 5290, 5288, 5290, 5300, 5295, 5288, 5292, 5285, 5296, 5290, 5289, 5295]
[5346, 5357, 5350, 5351, 5347, 5352, 5351, 5353, 5345, 5355, 5346, 5357, 5350, 5341, 5355]
Simulated Optimal Coverage/Tuatara genes (ASM311381v1)_simulation/Canon Site C/100.txt found, skipping to next file
Simulated Optimal Coverage/Tuatara genes (ASM311381v1)_simulation/Canon Site C/200.txt found, skipping to next file
Simulated Optimal Coverage/Tuatara genes

[20866, 20842, 20868, 20868, 20854, 20856, 20842, 20846, 20858, 20834, 20848, 20825, 20842, 20868, 20829]
[21049, 21053, 21082, 21063, 21029, 21058, 21037, 21040, 21026, 21058, 21056, 21037, 21019, 21050, 21027]
[21200, 21174, 21187, 21190, 21201, 21183, 21177, 21189, 21186, 21171, 21162, 21176, 21178, 21192, 21160]
[21298, 21270, 21291, 21300, 21301, 21292, 21291, 21285, 21280, 21280, 21273, 21276, 21271, 21285, 21263]
[21388, 21361, 21389, 21389, 21393, 21387, 21385, 21369, 21376, 21375, 21361, 21363, 21361, 21385, 21351]
Simulated Optimal Coverage/Zebra finch genes (bTaeGut1_v1.p)_simulation/Canon Site C/100.txt found, skipping to next file
Simulated Optimal Coverage/Zebra finch genes (bTaeGut1_v1.p)_simulation/Canon Site C/200.txt found, skipping to next file
Simulated Optimal Coverage/Zebra finch genes (bTaeGut1_v1.p)_simulation/Canon Site C/300.txt found, skipping to next file
Simulated Optimal Coverage/Zebra finch genes (bTaeGut1_v1.p)_simulation/Canon Site C/400.txt found, skip

[10391, 10402, 10402, 10394, 10398, 10388, 10393, 10360, 10360, 10402, 10372, 10377, 10360, 10376, 10391]
[10653, 10668, 10668, 10657, 10648, 10653, 10659, 10657, 10659, 10664, 10652, 10646, 10657, 10658, 10656]
[10853, 10872, 10852, 10858, 10850, 10864, 10846, 10855, 10870, 10860, 10851, 10832, 10864, 10847, 10856]
[11001, 11002, 10995, 11003, 11002, 10997, 10998, 10996, 10997, 11000, 10992, 10969, 10996, 10992, 11005]
[11120, 11124, 11110, 11113, 11126, 11122, 11121, 11116, 11122, 11126, 11113, 11110, 11116, 11114, 11118]
[11221, 11229, 11209, 11222, 11214, 11220, 11210, 11209, 11228, 11232, 11222, 11216, 11214, 11216, 11218]
Simulated Optimal Coverage/Opossum genes (ASM229v1)_simulation/Canon Site C/100.txt found, skipping to next file
Simulated Optimal Coverage/Opossum genes (ASM229v1)_simulation/Canon Site C/200.txt found, skipping to next file
Simulated Optimal Coverage/Opossum genes (ASM229v1)_simulation/Canon Site C/300.txt found, skipping to next file
Simulated Optimal Coverag

Simulated Optimal Coverage/Rabbit genes (OryCun2.0)_simulation/Canon Site C/0.txt found, skipping to next file
[5554, 5554, 5554, 5554, 5554, 5554, 5554, 5554, 5554, 5554, 5554, 5554, 5554, 5554, 5554]
[7427, 7427, 7434, 7427, 7427, 7435, 7427, 7427, 7427, 7427, 7427, 7427, 7427, 7427, 7427]
[8544, 8544, 8545, 8544, 8509, 8525, 8544, 8517, 8522, 8530, 8544, 8530, 8519, 8555, 8517]
[9203, 9203, 9185, 9208, 9207, 9197, 9204, 9203, 9203, 9206, 9199, 9202, 9214, 9198, 9194]
[9626, 9621, 9604, 9620, 9616, 9623, 9612, 9616, 9594, 9612, 9619, 9616, 9618, 9606, 9607]
[9922, 9909, 9931, 9922, 9939, 9916, 9909, 9912, 9908, 9915, 9911, 9912, 9919, 9921, 9908]
[10159, 10116, 10148, 10133, 10134, 10141, 10150, 10150, 10128, 10140, 10134, 10140, 10160, 10148, 10132]
[10347, 10330, 10325, 10348, 10320, 10332, 10332, 10342, 10324, 10333, 10337, 10330, 10341, 10341, 10323]
[10511, 10493, 10485, 10498, 10477, 10499, 10486, 10505, 10476, 10486, 10499, 10480, 10500, 10497, 10483]
Simulated Optimal Coverag

[40292, 40300, 40284, 40291, 40246, 40301, 40287, 40272, 40276, 40277, 40304, 40291, 40300, 40295, 40279]
[41023, 41018, 41015, 41018, 41000, 41040, 41036, 41022, 41015, 41029, 41087, 41040, 41021, 41042, 41034]
[41615, 41626, 41626, 41613, 41603, 41636, 41645, 41617, 41581, 41640, 41675, 41625, 41630, 41631, 41601]
[42106, 42110, 42108, 42110, 42107, 42129, 42136, 42124, 42090, 42135, 42170, 42094, 42133, 42126, 42097]
Simulated Optimal Coverage/Mouse genes (GRCm39)_simulation/Canon Site C/100.txt found, skipping to next file
Simulated Optimal Coverage/Mouse genes (GRCm39)_simulation/Canon Site C/200.txt found, skipping to next file
Simulated Optimal Coverage/Mouse genes (GRCm39)_simulation/Canon Site C/300.txt found, skipping to next file
Simulated Optimal Coverage/Mouse genes (GRCm39)_simulation/Canon Site C/400.txt found, skipping to next file
Simulated Optimal Coverage/Mouse genes (GRCm39)_simulation/Canon Site C/500.txt found, skipping to next file
Simulated Optimal Coverage/Mous

In [88]:
def simulate_plot(genome_filename, mature_filename, canon_site):
    #load total data from folder 'filename'
    total_data = {}
    new_file_path = "Simulated Optimal Coverage/"+genome_filename+"_simulation/Canon Site "+canon_site
    for ID in glob.glob(os.path.join(new_file_path, '*.txt')):
           with open(os.path.join(os.getcwd(), ID), 'r') as f:
                total_data[int(os.path.basename(ID).split('/')[-1].strip("%.tx"))] = (ast.literal_eval(f.read()))
                
    #sort total_data by number genes
    total_data = dict(sorted(total_data.items(), key=lambda x:x[1]))

    #generate means points (to plot line through boxplots)
    mean = {}
    maxes = {}
    mins = {}
    for i in total_data:
        mean[int(i)] = np.median(total_data[i])
        maxes[int(i)] = np.quantile(total_data[i],.95)
        mins[int(i)] = np.quantile(total_data[i],.05)
        
    #font
    font = {'family' : 'Helvetica',
        'weight' : 'bold',
        'size'   : 12}
    plt.rc('font', **font)
    
    #figure
    fig,ax = plt.subplots(figsize=(12,10))
    fig.patch.set_facecolor('white')
    plt.title(genome_filename + " Canon Site " + canon_site +" Optimal Cover Boxplot")
    
    #boxplot
    ax.boxplot(total_data.values(), 
               positions=list(total_data.keys()), 
               widths = 40, 
               medianprops = dict(color="#ed5a31",linewidth=1.1))
    ax.set_xticklabels(total_data.keys())
    plt.xticks(np.arange(0, 1400, step=100))
    
    #median line
    plt.plot(mean.keys(), mean.values(), c = '#cb275a')
    plt.plot(maxes.keys(), maxes.values(), c = '#cb275a', alpha = .4)
    plt.plot(mins.keys(), mins.values(), c = '#cb275a', alpha = .4)
    plt.fill_between(list(mean.keys()), list(mean.values()), list(maxes.values()), facecolor='#cb275a', alpha = .4,interpolate=True)
    plt.fill_between(list(mean.keys()), list(mins.values()), list(mean.values()), facecolor='#cb275a', alpha = .4,interpolate=True)
    
    #plot real coverage of Norm_v, Pre_v, and Post_v
    new_file_path = "Real Coverage/"+mature_filename+"/Canon Site "+canon_site
    with open(new_file_path+"/Norm_v.txt", 'r') as f:
        norm_data = ast.literal_eval(f.read())
    plt.plot(norm_data[0], norm_data[1], marker="x", markersize=10, markeredgecolor="#cb275a")
    
    #legend
    plt.rcParams.update({'font.size': 12})
    patches = [mpatches.Patch(color='#cb275a', label='Combined')]
    plt.legend(handles=patches, bbox_to_anchor=(1.01, 1.0), loc='upper left')
    
    #zoom in
    plt.rcParams.update({'font.size': 12})
    plt.rcParams.update({'font.weight': 'normal'})
   
    y_margin = int(max(mean.values()))/18
    x_margin = 75
    x1, x2, y1, y2 = norm_data[0]-x_margin,norm_data[0]+x_margin,norm_data[1]-y_margin,norm_data[1]+y_margin
    d1 = ((.5**2)+(mins[700]/maxes[1300]**2))**.5
    beta = 1.4
    gamma = .07
    z = 13*d1
    axins1 = zoomed_inset_axes(ax, zoom = z, loc=7)
    
    axins1.plot(norm_data[0], norm_data[1], marker="x", markersize=10, markeredgecolor="#cb275a")
    axins1.boxplot(total_data.values(), 
               positions=list(total_data.keys()), 
               widths = 10, 
               medianprops = dict(color="#ed5a31",linewidth=1.1))
    axins1.plot(mean.keys(), mean.values(), c = '#cb275a')
    axins1.plot(maxes.keys(), maxes.values(),c = '#cb275a', alpha = .4)
    axins1.plot(mins.keys(), mins.values(), c = '#cb275a', alpha = .4)
    axins1.fill_between(list(mean.keys()), list(mean.values()), list(maxes.values()), facecolor='#cb275a', alpha = .4,interpolate=True)
    axins1.fill_between(list(mean.keys()), list(mins.values()), list(mean.values()), facecolor='#cb275a', alpha = .4,interpolate=True)
    
    axins1.set_xlim(x1, x2)
    axins1.set_ylim(y1, y2)
    mark_inset(ax, axins1, loc1=1, loc2=3, fc="none", ec="0.5")
    
    #save plot
    if not os.path.exists('Simulated Plots'):
        os.makedirs('Simulated Plots')
    plt.savefig("Simulated Plots/"+genome_filename + " Canon Site " + canon_site +" Optimal Cover Boxplot.png")
    plt.show()

In [ ]:
species = 2
simulate_plot(species_list[species][0], species_list[species][1], 'A')

In [ ]:
#run simulate_plot() here
sites = ['A','B','C']
for species in species_list:
    for site in sites:
        simulate_plot(species[0], species[1], site)